# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [104]:
import requests
import re
from bs4 import BeautifulSoup

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [105]:
input_keyword = "除數函數"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
#print(utf8_url)
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
#print(utf8_url)
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

除數函數: %E9%99%A4%E6%95%B8%E5%87%BD%E6%95%B8
/wiki/%E9%99%A4%E6%95%B8%E5%87%BD%E6%95%B8


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [106]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': root_keyword_link, ##需要wiki??
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
story=""
for paragraph in content:
    print(paragraph.get_text())
    #story=story+paragraph.get_text()

#print(story)

在數論上，除數函數是一類算術函數。

除數函數




σ

x


(
n
)


{\displaystyle \sigma _{x}(n)}

定義為n的正因數的



x


{\displaystyle x}

次冪之和，即

其中一些特殊情況：



### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [107]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [數論] /wiki/%E6%95%B8%E8%AB%96
外部連結: [算術函數] /wiki/%E7%AE%97%E8%A1%93%E5%87%BD%E6%95%B8


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [111]:
def WikiArticle(key_word_link, key_word, recursive):
    
    
    key_word=key_word.replace("/","")
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        '''
        
        Your code here
        
        '''
        path = "C:/Users/Yang/HW/D16_Wiki的爬蟲實作練習/WikiArticle"
        save_file = path + "/rec" + str(recursive) + "__" + str(key_word) + ".txt"
        story=""
        #for paragraph in content:
        #    story=story+paragraph.get_text()
        
        #with open(save_file, "w",encoding="utf-8") as text_file:
        #    text_file.write(story)

        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        '''
        
        Your code here
        
        '''
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            #print(a_tag)
            if(len(a_tag)>0):
                for links in a_tag:
                    dict_T = {links['href']:links.get_text()}
                    external_link_dict.update(dict_T)
                    #print(dict_T)
        #print(external_link_dict)
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            recursive = recursive + 1  # 遞迴深度加1
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
            #print(external_link_dict.items())

### 執行前個步驟定義好的爬蟲主程式

In [112]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E9%99%A4%E6%95%B8%E5%87%BD%E6%95%B8 (除數函數)
遞迴層[1] - /wiki/%E6%95%B8%E8%AB%96 (數論)
遞迴層[2] - /wiki/%E7%BA%AF%E7%B2%B9%E6%95%B0%E5%AD%A6 (純粹數學)
遞迴層[2] - /wiki/%E6%95%B4%E6%95%B0 (整數)
遞迴層[2] - /wiki/%E8%B3%AA%E6%95%B8 (質數)
遞迴層[2] - /wiki/%E5%90%88%E6%95%B0 (合數)
遞迴層[2] - /wiki/1 (1)
遞迴層[2] - /wiki/%E5%93%A5%E5%BE%B7%E5%B7%B4%E8%B5%AB%E7%8C%9C%E6%83%B3 (哥德巴赫猜想)
遞迴層[2] - /wiki/%E5%AD%AA%E7%94%9F%E8%B4%A8%E6%95%B0%E7%8C%9C%E6%83%B3 (孿生質數猜想)
遞迴層[2] - /wiki/%E6%9C%89%E7%90%86%E6%95%B8 (有理數)
遞迴層[2] - /wiki/%E4%BB%A3%E6%95%B8%E6%95%B4%E6%95%B8 (代數整數)
遞迴層[2] - /wiki/%E4%B8%9F%E7%95%AA%E5%9C%96%E6%96%B9%E7%A8%8B (丟番圖方程)
遞迴層[2] - /wiki/%E8%A7%A3%E6%9E%90%E5%87%BD%E6%95%B8 (解析函數)
遞迴層[2] - /wiki/%E9%BB%8E%E6%9B%BC%CE%B6%E5%87%BD%E6%95%B8 (黎曼ζ函數)
遞迴層[2] - /wiki/%E4%B8%9F%E7%95%AA%E5%9C%96%E9%80%BC%E8%BF%91 (丟番圖逼近)
遞迴層[2] - /wiki/%E7%AE%97%E8%A1%93 (算術)
遞迴層[2] - /wiki/%E6%88%88%E5%BC%97%E9%9B%B7%C2%B7%E5%93%88%E7%BD%97%E5%BE%B7%C2%B7%E5%93%88%E4%BB%A3 (戈弗雷·哈羅德·哈代)
遞迴層[2] - /wiki/%E6%84%9B